## Notebook : 04_evaluation_and_metrics.ipynb
-  Objectif : Évaluer tous les modèles entraînés (UNet Mini, VGG16, MobileNetV2...) sur le test set

# 1 - Imports, GPU & vérifications préliminaires
## 1.1 - Librairies

In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib
import time
import hashlib
import docx

In [ ]:
print("🧠 TensorFlow version :", tf.__version__)
print("🔍 GPU disponible :", tf.config.list_physical_devices('GPU'))
print("🖥️ Détails des devices :")
print(device_lib.list_local_devices())

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

In [ ]:
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ Allocation mémoire GPU dynamique activée")
    except RuntimeError as e:
        print(f"⚠️ Erreur d'initialisation GPU : {e}")
else:
    print("❌ Aucun GPU détecté — exécution sur CPU.")

## 1.2 - Ajout de src/ au PYTHONPATH

In [ ]:
project_root = Path(".." ).resolve()
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

## 1.3 - Définition des chemins

In [ ]:
models_dir = project_root / "models"
data_path = project_root / "data" / "processed" / "augmented" / "test.npz"
outputs_metrics = project_root / "outputs" / "metrics"
outputs_figures = project_root / "outputs" / "figures"

outputs_metrics.mkdir(parents=True, exist_ok=True)
outputs_figures.mkdir(parents=True, exist_ok=True)

# 2 - Vérifications préliminaires
## 2.1 - Check chemins critiques

In [ ]:
from utils.guardrail import check_paths_exist, check_imports
check_paths_exist([models_dir, data_path])

## 2.2 - Check imports

In [ ]:
check_imports(["model_training.metrics", "utils.viz_utils"])

from model_training.metrics import iou_score, dice_coef
from utils.viz_utils import show_image_mask_grid_overlay

# 3 - Chargement du jeu de test

In [ ]:
test_data = np.load(data_path)
X_test, Y_test = test_data["X"], test_data["Y"]
print(f"✅ Jeu de test chargé : {X_test.shape}, {Y_test.shape}")

# 4 - Évaluation des modèles entraînés
## 4.1 - Chargement et évaluation des modèles

In [ ]:
model_files = list(models_dir.glob("*.h5"))

In [ ]:
results = []

In [ ]:
for model_path in model_files:
    model_name = model_path.stem
    print(f"\n🔍 Évaluation du modèle : {model_name}")
    try:
        model = keras.models.load_model(model_path, custom_objects={"iou_score": iou_score, "dice_coef": dice_coef})
    except Exception as e:
        print(f"⚠️ Erreur de chargement : {e}")
        continue

    start = time.time()
    metrics = model.evaluate(X_test, Y_test, verbose=0)
    end = time.time()

    duration = end - start
    model_size = model_path.stat().st_size / (1024 * 1024)  # MB
    param_count = model.count_params()
    hash_val = hashlib.sha256(model_path.read_bytes()).hexdigest()

    results.append({
        "model": model_name,
        "val_loss": metrics[0],
        "accuracy": metrics[1],
        "iou_score": metrics[2],
        "dice_coef": metrics[3],
        "inference_time": round(duration / len(X_test), 4),
        "model_size_MB": round(model_size, 2),
        "params": param_count,
        "hash": hash_val[:10]  # extrait pour suivi
    })

# 5 - Synthèse et visualisations
## 5.1 - Tableau comparatif

In [ ]:
results_df = pd.DataFrame(results).set_index("model")
display(results_df.sort_values(by="iou_score", ascending=False))

## 5.2 - Export CSV

In [ ]:
results_df.to_csv(outputs_metrics / "evaluation_scores.csv")

## 5.3 - Heatmap comparative

In [ ]:
plt.figure(figsize=(10, 5))
sns.heatmap(results_df[["iou_score", "dice_coef", "accuracy"]], annot=True, fmt=".3f", cmap="Blues")
plt.title("Heatmap des performances modèles")
plt.tight_layout()
plt.savefig(outputs_figures / "heatmap_evaluation_scores.png")
plt.show()

# 6 - Visualisation qualitative
## 6.1 - Choix du meilleur modèle

In [ ]:
best_model_name = results_df.sort_values(by="iou_score", ascending=False).index[0]
print(f"🏆 Meilleur modèle détecté : {best_model_name}")
best_model_path = models_dir / f"{best_model_name}.h5"
best_model = keras.models.load_model(best_model_path, custom_objects={"iou_score": iou_score, "dice_coef": dice_coef})

## 6.2 - Visualisation (5 exemples)

In [ ]:
for i in range(5):
    pred = best_model.predict(X_test[i:i+1])
    pred_mask = np.argmax(pred[0], axis=-1)

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes[0].imshow(X_test[i])
    axes[0].set_title("Image")
    axes[1].imshow(Y_test[i], cmap="nipy_spectral")
    axes[1].set_title("Mask Réel")
    axes[2].imshow(pred_mask, cmap="nipy_spectral")
    axes[2].set_title("Prédiction")
    for ax in axes:
        ax.axis('off')
    plt.tight_layout()
    plt.savefig(outputs_figures / f"prediction_sample_{i}.png")
    plt.show()

# 7 - Génération d'un rapport DOCX
## 7.1 - Création du document

In [ ]:
from docx import Document
from docx.shared import Inches

In [ ]:
doc = Document()
doc.add_heading("Rapport d'évaluation P08 - Segmentation", level=1)
doc.add_paragraph(f"Modèle sélectionné : {best_model_name}")
doc.add_paragraph(f"Paramètres : {results_df.loc[best_model_name]['params']} / Taille : {results_df.loc[best_model_name]['model_size_MB']} Mo")
doc.add_paragraph(f"Hash du modèle : {results_df.loc[best_model_name]['hash']}")

In [ ]:
doc.add_heading("Scores", level=2)
for metric in ["accuracy", "iou_score", "dice_coef"]:
    doc.add_paragraph(f"{metric} : {results_df.loc[best_model_name][metric]:.4f}")

In [ ]:
doc.add_picture(str(outputs_figures / "heatmap_evaluation_scores.png"), width=Inches(5.5))

In [ ]:
for i in range(5):
    img_path = outputs_figures / f"prediction_sample_{i}.png"
    if img_path.exists():
        doc.add_picture(str(img_path), width=Inches(5.5))

In [ ]:
doc.save(outputs_metrics / "rapport_performance.docx")
print("📄 Rapport DOCX généré avec succès.")